<a href="https://colab.research.google.com/github/Kimuksung/bigdata/blob/master/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""

DNN model
    - hidden layer : relu function
    - output layer : Softmax activation function
    - one hidden layer classifier
    - node : 5
    - data set : iris
    
"""

import tensorflow.compat.v1 as tf # ver1.x
tf.disable_v2_behavior() # ver2.0 사용안함
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# x, y data
iris = load_iris() 
x_data = iris.data
y_data = iris.target
y_data = y_data.reshape(-1,1)

obj = OneHotEncoder()
# sparse -> numpy
y_data = obj.fit_transform(y_data).toarray()
y_data.shape

# x, y 변수 정의
X = tf.placeholder(dtype = tf.float32 , shape =[None , 4])
Y = tf.placeholder(dtype = tf.float32 , shape =[None , 3])

# =============================================================================
# Dnn network
# =============================================================================

hidden_node = 5

# hidden layer
w1 = tf.Variable(tf.random_normal([ 4 , hidden_node]))
b1 = tf.Variable(tf.random_normal([ hidden_node ]))

# output layer 
w2 = tf.Variable(tf.random_normal([ hidden_node , 3]))
b2 = tf.Variable(tf.random_normal([ 3 ]))

# softmax 분류기 
hidden_output = tf.nn.relu(tf.matmul(X, w1) + b1) # hidden layer 

model = tf.matmul(hidden_output , w2) + b2                           
                           
# softmax(예측치)
softmax = tf.nn.softmax(model) 

# loss function : Entropy 이용 : -sum(Y * log(model)) 
# softmax  + crossentrophy
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    labels = Y , logits = model
    ))

# optimizer : 오차 최소화(w, b update) 
train = tf.train.AdamOptimizer(0.1).minimize(loss) # 오차 최소화

# argmax() : encoding -> decoding
y_pred = tf.argmax(softmax , axis =1)
y_true = tf.argmax(Y , axis =1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    feed_data= { X : x_data , Y : y_data}
    
    #반복 학습
    for step in range(500):
        _, loss_val = sess.run([train , loss] , feed_dict = feed_data)
        
        if (step+1)% 50 == 0 :
            print(" step : {} , loss : {}".format(step+1 , loss_val))

    y_pred_re = sess.run( y_pred , feed_dict ={ X :x_data })
    y_true_re = sess.run(y_true , feed_dict = { Y : y_data })
    
    print("y pred = " , y_pred_re)
    print("y true = " , y_true_re)
    acc = accuracy_score(y_true_re,y_pred_re)
    print("acc = " , acc)

Instructions for updating:
non-resource variables are not supported in the long term
 step : 50 , loss : 0.4927338659763336
 step : 100 , loss : 0.16919703781604767
 step : 150 , loss : 0.09525299817323685
 step : 200 , loss : 0.07512073963880539
 step : 250 , loss : 0.06545151770114899
 step : 300 , loss : 0.05962446331977844
 step : 350 , loss : 0.05574227496981621
 step : 400 , loss : 0.05301323160529137
 step : 450 , loss : 0.051008980721235275
 step : 500 , loss : 0.049476031213998795
y pred =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
y true =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [2]:
import tensorflow.compat.v1 as tf # ver1.x
tf.disable_v2_behavior() # ver2.0 사용안함
from sklearn.datasets import load_wine # data set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score
import numpy as np
# 1. wine data load
wine = load_wine()

# 2. 변수 선택/전처리  
x_data = wine.data # 178x13
y_data = wine.target # 3개 domain
print(y_data) # 0-2
print(x_data.shape) # (178, 13)

# x_data : 정규화 
x_data = minmax_scale(x_data) # 0~1

# y변수 one-hot-encoding : 0=[1,0,0] / 1=[0,1,0] / 2=[0,0,1]
num_class = np.max(y_data)+1 # 2+1
print(num_class) # 3

y_data = np.eye(num_class)[y_data]
print(y_data.shape) # (178, 3)

# 4. train/test split
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=123)

# 5. X,Y 변수 정의  
X = tf.placeholder(tf.float32, shape=[None, 13]) # [n, 13개 원소]
Y = tf.placeholder(tf.float32, shape=[None, 3]) # [n, 3개 원소]

# 6. Hypter parameters
learning_rate = 0.01
iter_size = 1000
    
##############################
### DNN network
# =============================================================================
#   <조건1>   
#    - Hidden layer : relu()함수 이용  
#    - Output layer : softmax()함수 이용 
#    - 2개의 은닉층을 갖는 DNN 분류기
#      hidden1 : nodes = 6
#      hidden2 : nodes = 3  
#   <조건2> hyper parameters
#     learning_rate = 0.01
#     iter_size = 1,000
#   <조건3>  
#     train/test(80:20)
#     x_data : 정규화 
#     y_data : one-hot encoding
# =============================================================================
##############################

hidden_node1 = 6
hidden_node2 = 3

# hidden layer1
w1 = tf.Variable(tf.random_normal([ 13 , hidden_node1]))
b1 = tf.Variable(tf.random_normal([ hidden_node1 ]))

# hidden layer2
w2 = tf.Variable(tf.random_normal([ hidden_node1 , hidden_node2]))
b2 = tf.Variable(tf.random_normal([ hidden_node2 ]))

# output layer 
w3 = tf.Variable(tf.random_normal([ hidden_node2 , 3]))
b3 = tf.Variable(tf.random_normal([ 3 ]))

hidden_output1 = tf.nn.relu(tf.matmul(X, w1) + b1) # hidden layer 
hidden_output2 = tf.nn.relu(tf.matmul(hidden_output1, w2) + b2)
model = tf.matmul(hidden_output2 , w3) + b3     
softmax = tf.nn.softmax(model) 


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    labels = Y , logits = model
    ))
train = tf.train.AdamOptimizer(0.1).minimize(loss)

y_pred = tf.argmax(softmax , axis =1)
y_true = tf.argmax(Y , axis =1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    feed_data= { X : x_train , Y : y_train}
    
    #반복 학습
    for step in range(500):
        _, loss_val = sess.run([train , loss] , feed_dict = feed_data)
        
        if (step+1)% 50 == 0 :
            print(" step : {} , loss : {}".format(step+1 , loss_val))

    # model result
    y_pred_re = sess.run( y_pred , feed_dict ={ X :x_test })
    y_true_re = sess.run(y_true , feed_dict = { Y : y_test })
    
    print("y pred = " , y_pred_re)
    print("y true = " , y_true_re)
    acc = accuracy_score(y_true_re,y_pred_re)
    print("acc = " , acc)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
(178, 13)
3
(178, 3)
 step : 50 , loss : 0.11532185971736908
 step : 100 , loss : 0.007790032308548689
 step : 150 , loss : 0.0038279995787888765
 step : 200 , loss : 0.002464162651449442
 step : 250 , loss : 0.0017523118294775486
 step : 300 , loss : 0.0013253113720566034
 step : 350 , loss : 0.0010461151832714677
 step : 400 , loss : 0.000850092910695821
 step : 450 , loss : 0.0007073901942931116
 step : 500 , loss : 0.0005998757551424205
y pred =  [2 1 2 1 1 2 0 2 2 1 2 2 2 0 0 2 1 1 0 1 1 2 2 2 1 2 2 1 0 0 0 0 1 1 2 1]
y true =  [2 1 2 1 1 2 0 2 2 1 2 2 2 0 0 2 1 1 0 1 2 2 2 2 1 2 2 1 0 0 0 0 2 1 2 1]
acc =  0.9444444444444444

In [3]:
import tensorflow.compat.v1 as tf # ver 1.x
tf.disable_v2_behavior() # ver 2.x 사용안함
from sklearn.preprocessing import OneHotEncoder # y data -> one hot
from sklearn.metrics import accuracy_score
import numpy as np

tf.set_random_seed(123) # w,b seed

# 1. MNIST dataset load
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# train set shape 확인 
x_train.shape # (60000, 28, 28) -> image(입력) : (size, h, w)
y_train.shape # (60000,) -> label(정답)

# test set shape
x_test.shape # (10000, 28, 28) -> image(입력)
y_test.shape # (10000,) -> label(정답)

# 2. 전처리 : X변수 정규화, Y변수 one-hot encoding  
x_train_nor, x_test_nor = x_train / 255.0, x_test / 255.0

# one-hot encoding
obj = OneHotEncoder()
train_labels = obj.fit_transform(y_train.reshape([-1, 1])).toarray()
train_labels.shape # (60000, 10)

test_labels = obj.fit_transform(y_test.reshape([-1, 1])).toarray()
test_labels.shape # (10000, 10)


# 3. 공급 data : image reshape(3d -> 2d)
train_images = x_train_nor.reshape(60000, 784)
test_images = x_test_nor.reshape(10000, 784)
train_images.shape # (60000, 784)
test_images.shape # (10000, 784)


X = tf.placeholder(tf.float32, [None, 784]) # [관측치, 입력수]
Y = tf.placeholder(tf.float32, [None, 10]) # [관측치, 출력수]

# hyper parameters
lr = 0.01
epochs = 15 # 전체 images(60,000) 20번 재사용 
batch_size = 100 # 1회 data 공급 size
iter_size = 600 # 반복횟수 

##############################
### MNIST DNN network
##############################
# =============================================================================
#   조건1> hyper parameters
#     learning_rate = 0.01
#     training_epochs = 15
#     batch_size = 100
#     iter_size = 600
#   조건2> DNN layer
#     Layer1 =  784 x 512
#     Layer2 =  512 x 256
#     Layer3 =  256 x 10 
# =============================================================================

input_size = train_images.shape[1]
hidden_node1 = 512
hidden_node2 = 256
output_size =10

# hidden layer1
w1 = tf.Variable(tf.random_normal([ input_size , hidden_node1]))
b1 = tf.Variable(tf.random_normal([ hidden_node1 ]))

# hidden layer2
w2 = tf.Variable(tf.random_normal([ hidden_node1 , hidden_node2]))
b2 = tf.Variable(tf.random_normal([ hidden_node2 ]))

# hidden layer3
w3 = tf.Variable(tf.random_normal([ hidden_node2 , hidden_node2]))
b3 = tf.Variable(tf.random_normal([ hidden_node2 ]))

# output layer 
w4 = tf.Variable(tf.random_normal([ hidden_node2 , output_size]))
b4 = tf.Variable(tf.random_normal([ output_size ]))

hidden_output1 = tf.nn.relu(tf.matmul(X, w1) + b1) # hidden layer 
hidden_output2 = tf.nn.relu(tf.matmul(hidden_output1, w2) + b2)
hidden_output3 = tf.nn.relu(tf.matmul(hidden_output2, w3) + b3)
model = tf.matmul(hidden_output2 , w4) + b4                           
softmax = tf.nn.softmax(model) 

# (3) loss function : Softmaxt + Cross Entorpy
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels = Y, logits = model))

# (4) optimizer 
train = tf.train.AdamOptimizer(lr).minimize(loss) 

# (5) encoding -> decoding 
y_pred = tf.argmax(softmax, axis = 1)
y_true = tf.argmax(Y, axis = 1)

# 6. model training
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer()) # w, b 초기화 
    for epoch in range(epochs) : 
        tot_loss = 0
        for step in range(iter_size):
            idx = np.random.choice( a = train_labels.shape[0] ,size = batch_size , replace =False)
            feed_data = { X : train_images[idx], Y : train_labels[idx]}
            _, loss_val = sess.run([train , loss] , feed_dict = feed_data)
            
            tot_loss += loss_val
        #1epoch 종료
        avg_loss = tot_loss / iter_size
        print("epoch = {} , loss = {}".format(epoch, avg_loss))
        
    # model test 
    feed_data2 = {X : test_images, Y : test_labels}
    y_pred_re = sess.run(y_pred, feed_dict = feed_data2)
    y_true_re = sess.run(y_true, feed_dict = feed_data2)
    acc = accuracy_score(y_true_re, y_pred_re)
    print("accuracy =", acc)

11493376/11490434 [==============================] - 0s 0us/step
epoch = 0 , loss = 53.74276607250174
epoch = 1 , loss = 9.815601193227712
epoch = 2 , loss = 5.550342130366633
epoch = 3 , loss = 3.6091982772822604
epoch = 4 , loss = 2.929191029036694
epoch = 5 , loss = 2.157738314670073
epoch = 6 , loss = 2.1126319621956986
epoch = 7 , loss = 1.7146935196972064
epoch = 8 , loss = 1.3126343060453844
epoch = 9 , loss = 1.5064779489444613
epoch = 10 , loss = 1.1906491389160943
epoch = 11 , loss = 0.9195669878533951
epoch = 12 , loss = 0.8787473325132692
epoch = 13 , loss = 0.68781101511618
epoch = 14 , loss = 0.5746037287521939
accuracy = 0.9603


# DNN - tensorflow 1.x


*   입력층(input layer)과 출력층(output layer) 사이에 여러 개의 은닉층(hidden layer) 
*   중간층의 다층화로 뉴런 처리와 전달, 산출되는 특징 값이 늘어남 → 정확도 향상
*   파라미터 수가 너무 많아짐 → 연산 많아짐, 과적합
*   완전연결 계층(Fully Connected NN)
*  layer node 수 : 입력층과 가까울 수록 많게, 출력층과 가까울 수록 적게 지정




---


# activation  : 활성화 함수


1.   Sigmoid function

*   0~1 까지 연속적으로 변화하는 출력값을 갖기 때문에 가중치나 바이어스(bias) 변화 시 출력에 변화를 준다

*    이진 분류 문제에서 출력층에 주로 쓰입니다



2.   ReLU function
*    0보다 작을 때는 0을 사용, 0보다 큰 값에 대해서는 해당 값을 그대로 사용하는 방법

*    은익층에 주로 쓰입니다

3.   linear
*    입력뉴런과 가중치로 계산된 결과값이 그대로 출력

4.   softmax
*    다중 클래스 분류 문제에서 출력층에 주로 쓰입니다.
---
# Loss
* 현재 가중치 세트를 평가하는 데 사용한 손실 함수
---
# optimizer 
* 최적의 가중치를 검색하는 데 사용되는 최적화 알고리즘
* ex) adam / SGD / RMSprop ..
* loss를 이용해서 backpropagation을 해주어 가중치 조절

<a href="https://ibb.co/xgFyJwS"><img src="https://i.ibb.co/qgJzkhM/DNN.png" alt="DNN" border="0"></a>




---



참고 : https://tykimos.github.io/2017/01/27/MLP_Layer_Talk/